In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from torch.utils.data import DataLoader
from hcmus.core import appconfig
from hcmus.utils import viz_utils
from hcmus.lbs import LabelStudioConnector
from hcmus.data import CroppedObjectClassificationDataset

2025-06-16 21:45:48.912 | INFO     | hcmus.core.appconfig:<module>:7 - Load DotEnv: True
/Volumes/Cucumber/Projects/item-identification/.venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [3]:
import mlflow
experiment_name = "/PrototypicalNetwork"
experiment = mlflow.get_experiment_by_name(experiment_name)
experiment_id = None
if not experiment:
    experiment_id = mlflow.create_experiment(experiment_name)
else:
    experiment_id = experiment.experiment_id

In [4]:
accepted_labels = """
8934804026817-nuoc-tuong-maggi-dau-nanh-thanh-diu-chai-700ml
8934804020402-nuoc-tuong-maggi-dau-nanh-dam-dac-700ml
8936048470524-mi-tom-3-mien-gold-vi-bo-ham-rau-thom-75g
8934822201272-bia-heineken-silver-lon-250ml
5054563022592-kem-danh-rang-sensodyne-giam-e-buotlam-sang-rang-100g
8934868166344-nuoc-rua-tay-lifebouy-450ml
8934707029182-hat-nem-knorr-thit-than-xuong-ong-va-tuy-900g
8934822361211-bia-heineken-khong-con-lon-330ml
8850006932346-kem-danh-rang-colgate-maxfresh-huong-tra-xanh-225g-+-ban-chai-long-to
8850006932322-kem-danh-rang-colgate-maxfresh-huong-bac-ha-225g-+-ban-chai-long-to
6920354827792-kem-danh-rang-colgate-maxfresh-tinh-chat-than-tre-225g-+-ban-chai-long-to
8936136163314-nuoc-mam-nam-ngu-3-trong-1-750ml
8934563198152-mi-hao-hao-chay-vi-rau-nam-74gr
8935049501503-nuoc-ngot-coca-cola-vi-nguyen-ban-lon-320ml
8936048470036-mi-3-mien-mi-chay-la-da-65g-(goi)
8935001215028-tra-tea-break-milk-350ml
8935006532755-kem-rua-mat-acnes-vitamin-lam-sang-da,-mo-seo-tham-50g
8935018801641-khan-bep-da-nang-pulppy-classic-2-lop-loc-2-cuon
8935024123287-ca-phe-sua-hoa-tan-g7-3in1-hop-336g-(16g-x-21-goi)
8935006538634-dau-tay-trang-hada-labo-sach-sau-duong-am-toi-uu-200ml
8934840000093-nuoc-khoang-chanh-lemona-chai-500ml
8997035601321-nuoc-uong-bu-dien-giai-bo-sung-ion-pocari-sweat-chai-900ml
4897036693162-nuoc-tang-luc-monster-mango-loco-355ml
809939073648-soda-sua-huong-vi-chuoi-milkis-banana
8934804038131-tra-hoa-qua-nestea-hoa-tan-hop-144g-12-goi-x-12g
8936048471248-mi-tom-chua-cay-dac-biet-3-mien-gold-goi-75g
8934804027333-tra-chanh-nestea-hoa-tan-hop-195g-15-goi-x-13g
8936010530881-tra-chanh-cozy-hoa-tan-hop-240g-16-goi-x-15g
8935049510604-nuoc-ngot-sprite-chai-320ml
8934588672118-nuoc-ngot-pepsi-khong-calo-vi-chanh-320ml
8935049501572-nuoc-ngot-fanta-vi-cam-lon-320ml
8935049500445-nuoc-ngot-fanta-chai-1.5l
8936721790017-nuoc-loc-th-true-water-chai-500ml
8935049501718-nuoc-ngot-sprite-lon-320ml
8936193070013-tra-xanh-khong-do-500ml
8935137601122-tra-xanh-thai-nguyen-100g
8934564600883-tra-den-c2-vi-dua-luoi-chai-455ml
8935001214274-tra-bi-dao-wonderfarm-lon-310ml
8934588870552-tra-olong-teaplus-450ml
6920354836930-kem-danh-rang-colgate-optic-white-cong-nghe-sac-tim-100g
8935049501374-nuoc-ngot-cocacola-vi-nguyen-ban-chai-600ml
8935039570700-ca-phe-sua-robusta-birdy-lon-170ml
8934673701402-nuoc-dao-necta-vinamilk-hop-1l
8935005801012-nuoc-loc-lavie-chai-1-5l
8936079140021-ca-phe-sua-highland-coffee-lon-185ml
8934588640445-ca-phe-sua-boss-lon-180ml
8934683008614-ca-phe-sua-vinacafe-3-in-1-gold-tui-480g-24-goi-tui
8934683008867-nuoc-tang-luc-wake-up-247-vi-ca-phe-330ml
8935328600156-tra-xanh-tui-loc-phuc-long-2g-x-hop-25-goi
4902430805322-nuoc-rua-tay-diet-khuan-safeguard-trang-tinh-khiet-450ml
8936156730718-nuoc-xa-vai-blue-dam-dac-huong-thanh-xuan-tui-3.2l
8934868166825-dau-goi-sunsilk-natural-duong-ngan-gay-rung-650g
8934804040523-tra-vai-va-huong-hoa-lai-nestea-hoa-tan-hop-144g-12-goi-x-12g
8934822801335-bia-tiger-crystal-4-6-phan-tram-lon-330ml
8934868156871-dau-xa-sunsilk-mem-muot-dieu-ky-320g
8936010530713-tra-dao-cozy-hoa-tan-hop-16-goi-x-15g
8934868171850-nuoc-lau-kinh-sunlight-sieu-nhanh-sach-trong-suot-chai-520ml
8850228003541-nuoc-tang-luc-warrior-huong-dau-chai-330ml
8936193070075-nuoc-tang-luc-number-1-chai-330ml
8934588012112-nuoc-ngot-pepsi-cola-320ml
8936013254951-nuoc-rua-kinh-gift-540ml
8934868190394-nuoc-giat-comfort-huong-hoa-anh-dao-va-dao-tuoi-tui-3kg
3068320055008-nuoc-loc-evivan-chai-500ml
8936127794206-nuoc-uong-sua-trai-cay-th-true-juice-milk-huong-viet-quat-300ml
8935049511038-sua-trai-cay-nutriboost-cam-chai-1l
8934673573399-sua-tuoi-tiet-trung-co-duong,-100%-sua-tuoivinamilk-(1lit).
3760128640607-sua-tuoi-tiet-trung-promess-nguyen-kem-hop-1l
8934822212339-bia-heineken-lon-cao-330ml
8801382123446-nuoc-gao-han-quoc-loai-500ml
8936036201604-bia-hoegaarden-rosee-chai-248ml
8934692090013-bia-1664-blanc-5%-abv-chai-330ml
8934868166870-dau-goi-sunsilk-ong-muot-rang-ngoi-chai-650g
8934588233074-nuoc-tang-luc-sting-vi-dau-tay-do-330ml
8934822431211-bia-tiger-soju-infused-lager-wonder-melon-vi-dua-luoi-lon-330ml
8934822112332-bia-tiger-lon-cao-330ml-lon
8934868180470-dau-goi-dove-biotin-ngan-gay-rung-toc-880g
8935024170519-ca-phe-trung-nguyen-legend-classic-hop-357g-17g-x-21-goi
8934868166351-nuoc-rua-tay-lifebuoy-matcha-va-lo-hoichai-450g
8936191270057-ca-phe-cappuccino-dua-just-viet-hop-hop-10-goi-x-17g
8934868166924-dau-goi-sunsilk-mem-muot-dieu-ky-chai-650g
8934804028064-ca-phe-hoa-tan-nescafe-3in1-vi-nguyen-ban-tui-782g-(17g-x-46-goi)
8936122200276-ca-phe-mr.brown-blue-mountain-240ml
8934563122201-mi-hao-hao-vi-tom-chua-cay-goi-100g
8935136865709-sua-tam-purite-duong-da-mem-min-huong-hoa-hong-rose-chai-850ml
8934822451295-nuoc-trai-cay-len-men-strongbow-vi-thom-luu-3.5%-lon-330ml
8934868170990-nuoc-rua-tay-lifebuoy-vitamin-sua-duong-am-tui-1kg
8936094291005-bia-budweiser-5-lon-330ml
8934822121297-nuoc-trai-cay-len-men-strongbow-red-beries-4.5%-abv-lon-320ml
8934822111298-nuoc-trai-cay-len-men-strongbow-gold-apple-4-5-abv-lon-320ml
8935049501190-nuoc-tra-fuze-tea-vi-chanh-sa-450ml
8936199810026-cafe-the-coffee-house-sua-da-22gx10-goi-(hop)
8888589308920-tra-bi-dao-winter-melon-jj-lon-300ml
8934673101097-sua-tuoi-tiet-trung-tiet-trung-vinamilk-khong-duong-bich-220ml
8934563183158-mi-hao-hao-sa-te-hanh-tim-75g
8936048470012-mi-3-mien-tom-chua-cay-65g
8934683009925-ca-phe-sua-da-vinacafe-chat-hop-10-goi-24g
8850228007617-nuoc-tang-luc-redbull-250ml
8934563651138-mi-hao-hao,-vi-tom-chua-cayacecook,-coc-(67g)
8934588063060-nuoc-loc-aquafina-chai-1-5l
8935049510857-nuoc-loc-dasani-chai-1.5l"""
accepted_labels = accepted_labels.splitlines()
accepted_labels = [x for x in accepted_labels if len(x) > 0]

In [5]:
splits = {}
for split_name in ["train", "test", "val"]:
    lsb_connector = LabelStudioConnector(
        url=appconfig.LABEL_STUDIO_URL,
        api_key=appconfig.LABEL_STUDIO_API_KEY,
        project_id=appconfig.LABEL_STUDIO_PROJECT_MAPPING[split_name],
        temp_dir=appconfig.LABEL_STUDIO_TEMP_DIR
    )

    tasks = lsb_connector.get_tasks()
    labels = lsb_connector.extract_labels(tasks)
    dataset = lsb_connector.download_dataset(tasks, labels)
    dataset = lsb_connector.transform_labels(dataset, labels, accepted_labels)
    splits[split_name] = dataset

2025-06-16 21:45:50.824 | WARNING  | hcmus.lbs._label_studio_connector:get_tasks:144 - Page size is too large, only 3308 tasks available.
2025-06-16 21:45:50.824 | INFO     | hcmus.lbs._label_studio_connector:get_tasks:148 - New `page_to` applied: 34
Loading tasks: 100%|██████████| 34/34 [00:09<00:00,  3.69it/s]
2025-06-16 21:46:05.661 | WARNING  | hcmus.lbs._label_studio_connector:get_tasks:144 - Page size is too large, only 355 tasks available.
2025-06-16 21:46:05.661 | INFO     | hcmus.lbs._label_studio_connector:get_tasks:148 - New `page_to` applied: 4
Loading tasks: 100%|██████████| 4/4 [00:02<00:00,  1.47it/s]
2025-06-16 21:46:08.636 | WARNING  | hcmus.lbs._label_studio_connector:get_tasks:144 - Page size is too large, only 82 tasks available.
2025-06-16 21:46:08.636 | INFO     | hcmus.lbs._label_studio_connector:get_tasks:148 - New `page_to` applied: 1
Loading tasks: 100%|██████████| 1/1 [00:00<00:00,  1.75it/s]


In [6]:
import torchvision.transforms as transforms
transform_train = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomRotation(90),
    transforms.ToTensor(),
    # transforms.Normalize(
    #     mean=[0.485, 0.456, 0.406],
    #     std=[0.229, 0.224, 0.225]
    # )
])

transform_basic = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    # transforms.Normalize(
    #     mean=[0.485, 0.456, 0.406],
    #     std=[0.229, 0.224, 0.225]
    # )
])

In [7]:
import numpy as np
from torch.utils.data import DataLoader
from easyfsl.samplers import TaskSampler
from hcmus.data import CroppedFewShotDataset
device = "cpu"
skip_labels = [
    "object",
    "8936136163314-nuoc-mam-nam-ngu-3-trong-1-750ml",
    "8936079140021-ca-phe-sua-highland-coffee-lon-185ml",
    "8936048470012-mi-3-mien-tom-chua-cay-65g",
    "8934822111298-nuoc-trai-cay-len-men-strongbow-gold-apple-4-5-abv-lon-320ml",
    "8936036201604-bia-hoegaarden-rosee-chai-248ml",
    "8934673101097-sua-tuoi-tiet-trung-tiet-trung-vinamilk-khong-duong-bich-220ml",
    "8801382123446-nuoc-gao-han-quoc-loai-500ml",
    "5054563022592-kem-danh-rang-sensodyne-giam-e-buotlam-sang-rang-100g",
    "8850006932322-kem-danh-rang-colgate-maxfresh-huong-bac-ha-225g-+-ban-chai-long-to",
]
train_dataset = CroppedFewShotDataset(splits["train"], skip_labels=skip_labels, transforms=transform_train)
val_dataset = CroppedFewShotDataset(splits["val"], label2idx=train_dataset.label2idx, transforms=transform_basic, is_test=True)

epoch = 64
n_classes = len(train_dataset.label2idx)
n_way = 5
n_shot = 5
n_query = 10
n_tasks = 512
train_sampler = TaskSampler(
    dataset=train_dataset,
    n_way=n_way,
    n_shot=n_shot,
    n_query=n_query,
    n_tasks=n_tasks
)
val_sampler = TaskSampler(
    dataset=train_dataset,
    n_way=n_classes,
    n_shot=n_shot,
    n_query=0,
    n_tasks=n_tasks
)
train_dataloader = DataLoader(
    train_dataset,
    batch_sampler=train_sampler,
    collate_fn=train_sampler.episodic_collate_fn
)
val_support_dataloader = DataLoader(
    train_dataset,
    batch_sampler=val_sampler,
    collate_fn=val_sampler.episodic_collate_fn
)
val_dataloader = DataLoader(
    val_dataset,
    batch_size=128,
    shuffle=True
)

2025-06-16 21:46:09.353 | INFO     | hcmus.data._torch_dataset:__init__:156 - Auto infer `label2idx` mapping, mapping length: 90.


In [8]:
from clip import clip
from hcmus.models.prototype import setup_training
clip.available_models()

['RN50',
 'RN101',
 'RN50x4',
 'RN50x16',
 'RN50x64',
 'ViT-B/32',
 'ViT-B/16',
 'ViT-L/14',
 'ViT-L/14@336px']

In [9]:
backbone_name = "ViT-B/16"
freeze_backbone = False
model, trainer, optimizer, scheduler = setup_training(backbone_name, freeze_backbone)
val_support_iter = iter(val_support_dataloader)
query_images, query_labels = next(iter(val_dataloader))

TypeError: __init__() got an unexpected keyword argument 'feature_dim'

In [ ]:
from tqdm import tqdm

with mlflow.start_run(experiment_id=experiment_id, run_name="prototypical_clip_vit"):
    mlflow.log_params({
        "freeze_backbone": freeze_backbone
    })
    for step, episode in tqdm(enumerate(train_dataloader), desc="Training...", total=n_tasks):
        support_images, support_labels, query_images, query_labels, unique_labels = episode
        train_result = trainer.train_episode(support_images, support_labels, query_images, query_labels, optimizer)
        mlflow.log_metrics(train_result, step=step)

        support_images, support_labels, _, _, _ = next(val_support_iter)
        query_images, query_labels = next(iter(val_dataloader))
        val_result = trainer.evaluate_episode(support_images, support_labels, query_images, query_labels, True)
        mlflow.log_metrics(val_result, step=step)


Training...:  25%|██▌       | 25/100 [12:45<38:16, 30.62s/it]


🏃 View run prototypical_clip_vit at: http://jimica.ddns.net:5050/#/experiments/990966814347092903/runs/8ee0647761b24ff1a14bc94c97d261fa
🧪 View experiment at: http://jimica.ddns.net:5050/#/experiments/990966814347092903


KeyboardInterrupt: 